# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,muchkapskiy,51.8513,42.4717,-14.67,98,59,1.73,RU,1706472395
1,1,anadyr,64.7500,177.4833,-31.72,75,0,4.00,RU,1706472396
2,2,margaret river,-33.9500,115.0667,11.86,84,96,3.36,AU,1706472128
3,3,hotan,37.0997,79.9269,-2.68,45,100,1.11,CN,1706472396
4,4,vorgashor,67.5833,63.9500,-12.43,92,100,4.41,RU,1706472396


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.4,
    color = 'Country'
)

# Display the map
all_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
temp = city_data_df.loc[(city_data_df['Max Temp']<29) & (city_data_df['Max Temp']>18),:]
wind = temp.loc[temp['Wind Speed']<3,:]
cloud = wind.loc[wind['Cloudiness']==0,:]

# Drop any rows with null values
ideal = cloud.dropna()
# Display sample data
ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,hawaiian paradise park,19.5933,-154.9731,27.72,74,0,2.57,US,1706472398
29,29,laguna,38.4210,-121.4238,21.29,72,0,1.54,US,1706472402
106,106,el granada,37.5027,-122.4694,20.94,63,0,2.06,US,1706472423
121,121,santa maria del oro,25.9333,-105.3667,18.65,19,0,2.30,MX,1706472427
211,211,zvishavane,-20.3267,30.0665,21.82,56,0,1.28,ZW,1706472453
288,288,porto novo,6.4965,2.6036,26.63,52,0,2.64,BJ,1706472477
306,306,pacific grove,36.6177,-121.9166,23.56,55,0,2.06,US,1706472483
309,309,isla vista,34.4133,-119.8610,28.58,41,0,1.54,US,1706472469
340,340,adelaide,-34.9333,138.6000,21.83,70,0,1.54,AU,1706472293
514,514,broken hill,-31.9500,141.4333,22.63,35,0,2.48,AU,1706472543


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,hawaiian paradise park,US,19.5933,-154.9731,74,
29,laguna,US,38.4210,-121.4238,72,
106,el granada,US,37.5027,-122.4694,63,
121,santa maria del oro,MX,25.9333,-105.3667,19,
211,zvishavane,ZW,-20.3267,30.0665,56,
288,porto novo,BJ,6.4965,2.6036,52,
306,pacific grove,US,36.6177,-121.9166,55,
309,isla vista,US,34.4133,-119.8610,41,
340,adelaide,AU,-34.9333,138.6000,70,
514,broken hill,AU,-31.9500,141.4333,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row['Lat']
    lon=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty and resp to json format
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
el granada - nearest hotel: Beach House
santa maria del oro - nearest hotel: No hotel found
zvishavane - nearest hotel: Zvishavane Panorama Lodge
porto novo - nearest hotel: Hôtel 6500 CFA
pacific grove - nearest hotel: Pacific Grove Inn
isla vista - nearest hotel: The Club & Guest House
adelaide - nearest hotel: Quest - King William South
broken hill - nearest hotel: Ibis Styles
cuero - nearest hotel: Fairfield Inn & Suites by Marriott Cuero
bouafle - nearest hotel: Hôtel Krigan


,City,Country,Lat,Lng,Humidity,Hotel Name
11,hawaiian paradise park,US,19.5933,-154.9731,74,No hotel found
29,laguna,US,38.4210,-121.4238,72,Holiday Inn Express & Suites
106,el granada,US,37.5027,-122.4694,63,Beach House
121,santa maria del oro,MX,25.9333,-105.3667,19,No hotel found
211,zvishavane,ZW,-20.3267,30.0665,56,Zvishavane Panorama Lodge
288,porto novo,BJ,6.4965,2.6036,52,Hôtel 6500 CFA
306,pacific grove,US,36.6177,-121.9166,55,Pacific Grove Inn
309,isla vista,US,34.4133,-119.8610,41,The Club & Guest House
340,adelaide,AU,-34.9333,138.6000,70,Quest - King William South
514,broken hill,AU,-31.9500,141.4333,35,Ibis Styles


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)